In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from imutils import paths
import pandas as pd
import numpy as np
import pickle
import random
import os

In [56]:
# Variable Constants #
ORIG_INPUT_DATASET = "mask-classifier-images"
BASE_PATH = "dataset"

TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

CLASSES = ["Mask", "No_Mask"]

LE_PATH = os.path.sep.join(["output", "le.cpickle"])
BASE_CSV_PATH = "output"

MODEL_PATH = os.path.sep.join(["output", "model.cpickle"])

BATCH_SIZE = 32


In [64]:
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)
le = None

[INFO] loading network...


In [65]:
def openTrainList():
    with open('mask-classifier-images/imglist_train.txt', 'r') as file:
        lines = file.readlines()
        for idx, line in enumerate(lines):
            lines[idx] = line.split()
            
        df = pd.DataFrame(lines, columns=["imgpath", "label"])

    #assign series to X and Y
    X = df["imgpath"]
    Y = df["label"]

    #create training and validation split
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
    return X_train, X_val, Y_train, Y_val

#open the file for test image paths. creates similar df as for training data
def openTestList():
    with open('mask-classifier-images/imglist_test.txt') as file:
        lines = file.readlines()
        for idx, line in enumerate(lines):
            lines[idx] = line.split()
        
        df = pd.DataFrame(lines, columns=["imgpath", "label"])

    #assign series to X_test and Y_test    
    X_test, Y_test = df["imgpath"], df["label"]

    return X_test, Y_test

#creates dictionary of each split, nested with data
def makePathDict(X_train, Y_train, X_test, Y_test, X_val, Y_val):
    pathDict = {TRAIN:{'X':X_train,'Y':Y_train},
                TEST:{'X':X_test,'Y':Y_test},
                VAL:{'X':X_val,'Y':Y_val}}

    return pathDict



In [66]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = openTrainList()
X_test, Y_test = openTestList()
pathDict = makePathDict(X_train, Y_train, X_test, Y_test, X_val, Y_val)

In [67]:
for split, val in pathDict.items():
    print("[INFO] processing '{} split'...".format(split))
    p = os.path.sep.join([ORIG_INPUT_DATASET, BASE_PATH, split])
    imagePaths = list(paths.list_images(p))
    
    random.shuffle(imagePaths)
    labels = [p.split(os.path.sep)[-2] for p in imagePaths]
    if le is None:
        le = LabelEncoder()
        le.fit(labels)
    
    csvPath = os.path.sep.join([BASE_CSV_PATH, "{}.csv".format(split)])
    csv = open(csvPath, "w")
    
    for (b, i) in enumerate(range(0,len(imagePaths), BATCH_SIZE)):
            print("[INFO] processing batch {}/{}".format(b+1, int(np.ceil(len(imagePaths)/float(BATCH_SIZE)))))
            batchPaths = imagePaths[i:i + BATCH_SIZE]
            batchLabels = le.transform(labels[i:i + BATCH_SIZE])
            batchImages = []
            
            for imagePath in batchPaths:
                image = load_img(imagePath, target_size = (224, 224))
                image = img_to_array(image)
                
                image = np.expand_dims(image,axis=0)
                image = preprocess_input(image)
                
                batchImages.append(image)
                
            batchImages = np.vstack(batchImages)
            features = model.predict(batchImages, batch_size=BATCH_SIZE)
            features = features.reshape((features.shape[0], 7*7*512))
            
            for (label, vec) in zip(batchLabels, features):
                
                vec = ','.join([str(v) for v in vec])
                csv.write("{},{}\n".format(label, vec))
                
    csv.close()
    
with open(LE_PATH, "wb") as f:
    f.write(pickle.dumps(le))

[INFO] processing 'training split'...
[INFO] processing batch 1/212
[INFO] processing batch 2/212
[INFO] processing batch 3/212
[INFO] processing batch 4/212
[INFO] processing batch 5/212
[INFO] processing batch 6/212
[INFO] processing batch 7/212
[INFO] processing batch 8/212
[INFO] processing batch 9/212
[INFO] processing batch 10/212
[INFO] processing batch 11/212
[INFO] processing batch 12/212
[INFO] processing batch 13/212
[INFO] processing batch 14/212
[INFO] processing batch 15/212
[INFO] processing batch 16/212
[INFO] processing batch 17/212
[INFO] processing batch 18/212
[INFO] processing batch 19/212
[INFO] processing batch 20/212
[INFO] processing batch 21/212
[INFO] processing batch 22/212
[INFO] processing batch 23/212
[INFO] processing batch 24/212
[INFO] processing batch 25/212
[INFO] processing batch 26/212
[INFO] processing batch 27/212
[INFO] processing batch 28/212
[INFO] processing batch 29/212
[INFO] processing batch 30/212
[INFO] processing batch 31/212
[INFO] pro

[INFO] processing batch 50/67
[INFO] processing batch 51/67
[INFO] processing batch 52/67
[INFO] processing batch 53/67
[INFO] processing batch 54/67
[INFO] processing batch 55/67
[INFO] processing batch 56/67
[INFO] processing batch 57/67
[INFO] processing batch 58/67
[INFO] processing batch 59/67
[INFO] processing batch 60/67
[INFO] processing batch 61/67
[INFO] processing batch 62/67
[INFO] processing batch 63/67
[INFO] processing batch 64/67
[INFO] processing batch 65/67
[INFO] processing batch 66/67
[INFO] processing batch 67/67
[INFO] processing 'validation split'...
[INFO] processing batch 1/53
[INFO] processing batch 2/53
[INFO] processing batch 3/53
[INFO] processing batch 4/53
[INFO] processing batch 5/53
[INFO] processing batch 6/53
[INFO] processing batch 7/53
[INFO] processing batch 8/53
[INFO] processing batch 9/53
[INFO] processing batch 10/53
[INFO] processing batch 11/53
[INFO] processing batch 12/53
[INFO] processing batch 13/53
[INFO] processing batch 14/53
[INFO] pr

In [43]:
p = os.path.sep.join([ORIG_INPUT_DATASET,BASE_PATH, TRAIN, "Mask"])
list(paths.list_images(p))

[]

array(['Mask', 'No_Mask'], dtype='<U7')

In [55]:
! ls

coco.names

The system cannot find the path specified.



facewithmask.jpg
mask-classifier-images
models
output
pedestriantestimg.jpg
PlayAroundWithPretrainModels.ipynb
profpicalex.jpg
README.md
TransferLearning-FeatureExtraction.ipynb
